In [1]:
import cv2
import mahotas
import numpy as np
import joblib as jb
import pickle as pk
import seaborn as sns
import matplotlib.pyplot as plt

from os import listdir
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import Isomap
from sklearn.model_selection import train_test_split
from collections import Counter

from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.metrics import confusion_matrix

from sklearn.neural_network import MLPClassifier
from neuralNetwork.perceptron_qpso import perceptron 
from neuralNetwork.perceptron_pso import perceptron_pso
from neuralNetwork.perceptron_pso_bound import perceptron_pso_bound  

In [ ]:
#----------------
#Global variables
#----------------

root_path = './dataset2/'
#train_path = 'seg_train/seg_train/'
#test_path = 'seg_test/seg_test/'
#pred_path = 'seg_pred/seg_pred/'

# Dataset classes: buildings, forest, glacier, mountain, sea, street
data_clases = np.array(['cloudy', 
						'rain', 
						'shine', 
						'sunrise'])
IMG_HEIGHT = 150
IMG_WIDTH = 150

#---------------------------------
# Extract Global features
#---------------------------------
bins = 4 

# feature-descriptor-1: Hu Moments
def fd_hu_moments(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    feature = cv2.HuMoments(cv2.moments(image)).flatten()
    return feature

# feature-descriptor-2: Haralick Texture
def fd_haralick(image):
    # convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # compute the haralick texture feature vector
    haralick = mahotas.features.haralick(gray).mean(axis=0)
    # return the result
    return haralick

# feature-descriptor-3: Color Histogram
def fd_histogram(image, mask=None):
    # convert the image to HSV color-space
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # compute the color histogram
    hist  = cv2.calcHist([image], [0, 1, 2], None, [bins, bins, bins], [0, 256, 0, 256, 0, 256])
    # normalize the histogram
    cv2.normalize(hist, hist)
    # return the histogram
    return hist.flatten()

#---------------------------------
# prepare training and testing set
#---------------------------------

X_train_aux = []
y_train_aux = []

for c in range(len(data_clases)):
	for filename in listdir(root_path + data_clases[c] + '/'):
		# load image
		img_data = cv2.imread(root_path + data_clases[c] + '/' + filename) 
		img_data = cv2.resize(img_data, (IMG_HEIGHT, IMG_WIDTH))
        # Global Feature extraction
		fv_hu_moments = fd_hu_moments(img_data)
		#print(fv_hu_moments.shape)
		fv_haralick = fd_haralick(img_data)
		#print(fv_haralick.shape)        
		fv_histogram  = fd_histogram(img_data)
		#print(fv_histogram.shape)
		# Concatenate global features
		global_feature = np.hstack([fv_histogram, fv_haralick, fv_hu_moments])
		X_train_aux.append(global_feature)
		y_train_aux.append(c)
		print('> loaded %s %s' % (filename, img_data.shape))
		print('> global features %s %s' % (filename, global_feature.shape))

# convert to array
X_train_aux = np.array(X_train_aux)
y_train_aux = np.array(y_train_aux)

# normalize data
#X_train_aux = X_train_aux / 255

print('X_train shape: ', X_train_aux.shape)
print('y_train shape: ', y_train_aux.shape)

In [3]:
# normalize data (0,1)
t = MinMaxScaler()
t.fit(X_train_aux)
X_train_aux = t.transform(X_train_aux)

In [4]:
X_train_bal, X_test_bal, y_train_bal, y_test_bal = train_test_split(X_train_aux, y_train_aux, test_size=0.3, random_state=100)
print(f"Training target statistics: {Counter(y_train_bal)}")
print(f"Testing target statistics: {Counter(y_test_bal)}")

# Global variables
X_sample = len(X_train_bal)
X_input = len(X_train_bal[1])
X_class = len(np.unique(y_train_bal))

X_train = X_train_bal
y_train = y_train_bal
X_test = X_test_bal
y_test = y_test_bal

Training target statistics: Counter({3: 253, 0: 196, 2: 179, 1: 157})
Testing target statistics: Counter({0: 104, 3: 104, 2: 72, 1: 57})


# original features

## QDPSO

In [5]:
model_load = np.load('mcw_gBest_113_100_1000_84_qdpso.npy')
nn = perceptron(X_sample, X_input, X_class)

print("=====================================================================")
print("=====================================================================")
y_test_pred_load = np.argmax(nn.forward(X_test, model_load), axis=1)
cost_test_load = mean_squared_error(y_test, y_test_pred_load)
acc_test_load = accuracy_score(y_test, y_test_pred_load)
print('MSE: ', cost_test_load, ' ACC score: ', acc_test_load)


y_train_pred_load = np.argmax(nn.forward(X_train, model_load), axis=1)
cost_train_load = mean_squared_error(y_train, y_train_pred_load)
acc_train_load = accuracy_score(y_train, y_train_pred_load)
print('MSE: ', cost_train_load, ' ACC score: ', acc_train_load)
print("=====================================================================")
print("=====================================================================")

MSE:  0.20178041543026706  ACC score:  0.9169139465875371
MSE:  0.12611464968152866  ACC score:  0.9630573248407643


## PSO

In [6]:
model_load = np.load('mcw_gBest_113_100_1000_84_pso.npy')
nn = perceptron_pso(X_sample, X_input, X_class)

print("=====================================================================")
print("=====================================================================")
y_test_pred_load = np.argmax(nn.forward(X_test, model_load), axis=1)
cost_test_load = mean_squared_error(y_test, y_test_pred_load)
acc_test_load = accuracy_score(y_test, y_test_pred_load)
print('MSE: ', cost_test_load, ' ACC score: ', acc_test_load)


y_train_pred_load = np.argmax(nn.forward(X_train, model_load), axis=1)
cost_train_load = mean_squared_error(y_train, y_train_pred_load)
acc_train_load = accuracy_score(y_train, y_train_pred_load)
print('MSE: ', cost_train_load, ' ACC score: ', acc_train_load)
print("=====================================================================")
print("=====================================================================")

MSE:  0.45103857566765576  ACC score:  0.8516320474777448
MSE:  0.3375796178343949  ACC score:  0.8828025477707007


## PSO bound

In [7]:
model_load = np.load('mcw_gBest_113_100_1000_84_pso_bound.npy')
nn = perceptron_pso_bound(X_sample, X_input, X_class)

print("=====================================================================")
print("=====================================================================")
y_test_pred_load = np.argmax(nn.forward(X_test, model_load), axis=1)
cost_test_load = mean_squared_error(y_test, y_test_pred_load)
acc_test_load = accuracy_score(y_test, y_test_pred_load)
print('MSE: ', cost_test_load, ' ACC score: ', acc_test_load)


y_train_pred_load = np.argmax(nn.forward(X_train, model_load), axis=1)
cost_train_load = mean_squared_error(y_train, y_train_pred_load)
acc_train_load = accuracy_score(y_train, y_train_pred_load)
print('MSE: ', cost_train_load, ' ACC score: ', acc_train_load)
print("=====================================================================")
print("=====================================================================")

MSE:  0.35311572700296734  ACC score:  0.8724035608308606
MSE:  0.3681528662420382  ACC score:  0.8662420382165605


# 42 features

In [8]:
embedding = Isomap(n_components=42, n_neighbors=100)
X_transformed = embedding.fit_transform(X_train_aux)
print ('Shape of X_train transformed: ', X_transformed.shape)

X_train_bal, X_test_bal, y_train_bal, y_test_bal = train_test_split(X_transformed, y_train_aux, test_size=0.3, random_state=100)
print(f"Training target statistics: {Counter(y_train_bal)}")
print(f"Testing target statistics: {Counter(y_test_bal)}")

# Global variables
X_sample = len(X_train_bal)
X_input = len(X_train_bal[1])
X_class = len(np.unique(y_train_bal))

X_train = X_train_bal
y_train = y_train_bal
X_test = X_test_bal
y_test = y_test_bal

Shape of X_train transformed:  (1122, 42)
Training target statistics: Counter({3: 253, 0: 196, 2: 179, 1: 157})
Testing target statistics: Counter({0: 104, 3: 104, 2: 72, 1: 57})


## QDPSO

In [9]:
model_load = np.load('mcw_gBest_113_100_1000_42_qdpso.npy')
nn = perceptron(X_sample, X_input, X_class)

print("=====================================================================")
print("=====================================================================")
y_test_pred_load = np.argmax(nn.forward(X_test, model_load), axis=1)
cost_test_load = mean_squared_error(y_test, y_test_pred_load)
acc_test_load = accuracy_score(y_test, y_test_pred_load)
print('MSE: ', cost_test_load, ' ACC score: ', acc_test_load)


y_train_pred_load = np.argmax(nn.forward(X_train, model_load), axis=1)
cost_train_load = mean_squared_error(y_train, y_train_pred_load)
acc_train_load = accuracy_score(y_train, y_train_pred_load)
print('MSE: ', cost_train_load, ' ACC score: ', acc_train_load)
print("=====================================================================")
print("=====================================================================")

MSE:  0.2789317507418398  ACC score:  0.8724035608308606
MSE:  0.05222929936305733  ACC score:  0.9821656050955414


## PSO

In [10]:
model_load = np.load('mcw_gBest_113_100_1000_42_pso.npy')
nn = perceptron_pso(X_sample, X_input, X_class)

print("=====================================================================")
print("=====================================================================")
y_test_pred_load = np.argmax(nn.forward(X_test, model_load), axis=1)
cost_test_load = mean_squared_error(y_test, y_test_pred_load)
acc_test_load = accuracy_score(y_test, y_test_pred_load)
print('MSE: ', cost_test_load, ' ACC score: ', acc_test_load)


y_train_pred_load = np.argmax(nn.forward(X_train, model_load), axis=1)
cost_train_load = mean_squared_error(y_train, y_train_pred_load)
acc_train_load = accuracy_score(y_train, y_train_pred_load)
print('MSE: ', cost_train_load, ' ACC score: ', acc_train_load)
print("=====================================================================")
print("=====================================================================")

MSE:  0.258160237388724  ACC score:  0.8694362017804155
MSE:  0.2267515923566879  ACC score:  0.9299363057324841


## PSO bound

In [11]:
model_load = np.load('mcw_gBest_113_100_1000_42_pso_bound.npy')
nn = perceptron_pso_bound(X_sample, X_input, X_class)

print("=====================================================================")
print("=====================================================================")
y_test_pred_load = np.argmax(nn.forward(X_test, model_load), axis=1)
cost_test_load = mean_squared_error(y_test, y_test_pred_load)
acc_test_load = accuracy_score(y_test, y_test_pred_load)
print('MSE: ', cost_test_load, ' ACC score: ', acc_test_load)


y_train_pred_load = np.argmax(nn.forward(X_train, model_load), axis=1)
cost_train_load = mean_squared_error(y_train, y_train_pred_load)
acc_train_load = accuracy_score(y_train, y_train_pred_load)
print('MSE: ', cost_train_load, ' ACC score: ', acc_train_load)
print("=====================================================================")
print("=====================================================================")

MSE:  0.4035608308605341  ACC score:  0.8249258160237388
MSE:  0.37070063694267513  ACC score:  0.8624203821656051


# 14 Features

In [12]:
embedding = Isomap(n_components=14, n_neighbors=100)
X_transformed = embedding.fit_transform(X_train_aux)
print ('Shape of X_train transformed: ', X_transformed.shape)

X_train_bal, X_test_bal, y_train_bal, y_test_bal = train_test_split(X_transformed, y_train_aux, test_size=0.3, random_state=100)
print(f"Training target statistics: {Counter(y_train_bal)}")
print(f"Testing target statistics: {Counter(y_test_bal)}")

# Global variables
X_sample = len(X_train_bal)
X_input = len(X_train_bal[1])
X_class = len(np.unique(y_train_bal))

X_train = X_train_bal
y_train = y_train_bal
X_test = X_test_bal
y_test = y_test_bal

Shape of X_train transformed:  (1122, 14)
Training target statistics: Counter({3: 253, 0: 196, 2: 179, 1: 157})
Testing target statistics: Counter({0: 104, 3: 104, 2: 72, 1: 57})


## QDPSO

In [13]:
model_load = np.load('mcw_gBest_113_100_1000_14_pso.npy')
nn = perceptron_pso(X_sample, X_input, X_class)

print("=====================================================================")
print("=====================================================================")
y_test_pred_load = np.argmax(nn.forward(X_test, model_load), axis=1)
cost_test_load = mean_squared_error(y_test, y_test_pred_load)
acc_test_load = accuracy_score(y_test, y_test_pred_load)
print('MSE: ', cost_test_load, ' ACC score: ', acc_test_load)


y_train_pred_load = np.argmax(nn.forward(X_train, model_load), axis=1)
cost_train_load = mean_squared_error(y_train, y_train_pred_load)
acc_train_load = accuracy_score(y_train, y_train_pred_load)
print('MSE: ', cost_train_load, ' ACC score: ', acc_train_load)
print("=====================================================================")
print("=====================================================================")

MSE:  0.24925816023738873  ACC score:  0.8783382789317508
MSE:  0.2267515923566879  ACC score:  0.9210191082802548


## PSO

In [14]:
model_load = np.load('mcw_gBest_113_100_1000_14_pso.npy')
nn = perceptron_pso(X_sample, X_input, X_class)

print("=====================================================================")
print("=====================================================================")
y_test_pred_load = np.argmax(nn.forward(X_test, model_load), axis=1)
cost_test_load = mean_squared_error(y_test, y_test_pred_load)
acc_test_load = accuracy_score(y_test, y_test_pred_load)
print('MSE: ', cost_test_load, ' ACC score: ', acc_test_load)


y_train_pred_load = np.argmax(nn.forward(X_train, model_load), axis=1)
cost_train_load = mean_squared_error(y_train, y_train_pred_load)
acc_train_load = accuracy_score(y_train, y_train_pred_load)
print('MSE: ', cost_train_load, ' ACC score: ', acc_train_load)
print("=====================================================================")
print("=====================================================================")

MSE:  0.24925816023738873  ACC score:  0.8783382789317508
MSE:  0.2267515923566879  ACC score:  0.9210191082802548


## PSO bound

In [15]:
model_load = np.load('mcw_gBest_113_100_1000_14_pso_bound.npy')
nn = perceptron_pso_bound(X_sample, X_input, X_class)

print("=====================================================================")
print("=====================================================================")
y_test_pred_load = np.argmax(nn.forward(X_test, model_load), axis=1)
cost_test_load = mean_squared_error(y_test, y_test_pred_load)
acc_test_load = accuracy_score(y_test, y_test_pred_load)
print('MSE: ', cost_test_load, ' ACC score: ', acc_test_load)


y_train_pred_load = np.argmax(nn.forward(X_train, model_load), axis=1)
cost_train_load = mean_squared_error(y_train, y_train_pred_load)
acc_train_load = accuracy_score(y_train, y_train_pred_load)
print('MSE: ', cost_train_load, ' ACC score: ', acc_train_load)
print("=====================================================================")
print("=====================================================================")

MSE:  0.32047477744807124  ACC score:  0.8664688427299704
MSE:  0.3974522292993631  ACC score:  0.8509554140127389
